In [1]:
# 데이터셋과 시각화용 라이브러리
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_moons, make_circles, make_classification

In [2]:
# 데이터 분류와 모델 형성에 사용되는 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler         
from sklearn.neural_network import MLPClassifier

In [6]:
# 파라미터 설정
h = .02 # visualization을 위한 parameter / not for NN                       
alphas = np.logspace(-5,3,5) # NN의 alpha값
names = ['alpha' + str(i) for i in alphas] # 시각화시 plot의 title용

# use logspace : alphas - array([1.e-05, 1.e-03, 1.e-01, 1.e+01, 1.e+03]) 제약 지정
# names : ['alpha 1e-05', 'alpha 0.001', 'alpha 0.1', 'alpha 10.0', 'alpha 1000.0'] 

In [9]:
# alpha 값에 따른 neural network model 생성
classifiers = []
for i in alphas : 
    classifiers.append(MLPClassifier(solver = 'lbfgs',alpha = i, random_state = 1,
                                     hidden_layer_sizes = [100,100]))

In [15]:
# 데이터 생성
X, y = make_classification(n_features = 2, n_redundant = 0, n_informative = 2,
                           random_state = 0, n_clusters_per_class = 1) 
# pd.DataFrame(X).head()
# pd.DataFrame(y).head()

In [16]:
# uniform error 생성
rng = np.random.RandomState(2)
X += 2*rng.uniform(size = X.shape)
linearly_separable = (X,y)

In [17]:
# 여러 모양(원형, 선형)의 추가 데이터세트 생성
datasets = [make_moons(noise = 0.3, random_state = 0),
            make_circles(noise = 0.2, factor = 0.5, random_state = 1),
            linearly_separable]

In [18]:
# figure 생성
figure = plt.figure(figsize = (18,6))

<Figure size 1800x600 with 0 Axes>

In [19]:
## 3개의 데이터 구조를 5개의 alpha값에 따른 학습과 시각화

# 데이터세트 3종류, moon, circle, linear data 반복
for X,y in datasets : 
    X = StandardScaler().fit_transform(X)  
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 4)
    
    # grid 생성
    x_min, x_max = X[:, 0].min() - 5, X[:,0].max() +5
    y_min, y_max = X[:, 0].min() - 5, X[:,0].max() +5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # dataset 시각화를 위한 plot작업
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000','#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    
    # training point plot
    ax.scatter(X_train[:,0],X_train[:,1], c=y_train, cmap=cm_bright)
    
    # testing point plot - 여기서 alpha값은 sklearn의 알파와 다름, 투명도를 의미
    ax.scatter(X_test[:,0], X_train[:,1], c=y_train, cmap=cm_bright, alpha = 0.6)
    
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1
    
    # classifiers iterate - alpha값에 따른 5개의 분류기준 반복
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        
        # plot의 초평면(Decision Boundary)를 기준으로 다른 색을 할당
        
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot also the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='black', s=25)
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   alpha=0.6, edgecolors='black', s=25)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1        

ValueError: num must be 1 <= num <= 18, not 1000.0

<Figure size 640x480 with 0 Axes>